# COMP90042 Assignment #2: Cross-language Information Retreival

```Student Name: Yujie Cheng
Student ID: 724408```

Please do not edit this field. It must be left as is for use in marking.

## General info

**Due date**: 5pm, Wed 25th May

**Submission method**: see LMS

**Submission materials**: completed copy of this ipython notebook. *Do not upload the corpora*, you can assume these files will be extracted into the same folder as the code.  *Do not use absolute paths* such as paths to files on your machine, use *relative paths* instead. *Include output* in the ipython notebook, and ensure your code can run sucessfully from scratch (e.g., use *Kernel -> Restart and Run All* menu option to test.)

**Late submissions**: -10% per day, no late submissions after the first week

**Marks**: 25% of mark for class

**Overview**: For this project, you'll be building a cross language information retreival system (CLIR), which is capable of searching for important text documents written in a English given a query in German, and displaying the results in German. This incorporates machine translation, information retrieval using a vector space model, and IR evaluation. A key focus of this project is critical analysis and experimental evaluation, for which you will need to report on the relative merits of various options. 

**Materials**: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages as used for workshops and the previous assignments. The main part of this assignment should be done entirely in python, using built-in python packages and the packages used in the class (NLTK, Numpy, Scipy, Matplotlib, Sci-kit Learn, and Gemsim). Please do no use any other 3rd party packages. Note that some choices of the *extended part* of the assignment may require you to work with other tools, such as 3rd party translation tools or retreival engines, in which case this part of the assignment will require working in another environment, although you should report your results in the ipython notebook and attach your other files (code, shell scripts etc) with your submission.   

You are encouraged to make use of the iPython notebooks released for this class as well as other online documentation to guide your responses, but you should not copy directly from any source. Adapting code from the class notebooks is permitted.

**Data**: The other data you will need are:
  - parallel German-English corpus.  This data comes from the Europarl corpus, which is a collection of debates held in the EU parliament over many years, translated into several EU languages. This data was collated as part of the annual WMT translation competitions. The data has been tokenised, sentence aligned using the Church-Gale algorithm. 
  - CLIR evaluation corpus, comprising a large collection of English documents sourced from Wikipedia along with several German query strings and relevance judgements for each document and query pair. 
  
The data files are as follows:
  - *bitext-small.{en,de}* is the sentence-aligned parallel corpus, of a small enough size for you to use for developing word-alignment tools. You may wish to work on a smaller subset during code development.
  - *bitext-large.{en,de}* is a much larger version of the above corpus, for language modelling and translation.
  - *newstest2013.{en,de}* is a separate small parallel corpus reserved for evaluation.
  - *dev.{docs,queries,qrel}* is a set of documents in English, queries in German and query relevance judgements, for the development of the IR components and evaluation.
  
For details of the datasets, please see:
> Philipp Koehn. *Europarl: A Parallel Corpus for Statistical Machine Translation* MT Summit 2005.   

and
> Shigehiko Schamoni, Felix Hieber, Artem Sokolov, Stefan Riezler. *Learning Translational and Knowledge-based Similarities from Relevance Rankings for Cross-Language Retrieval* ACL 2014.

You can find the PDFs for both papers online with a quick web search.

**Evaluation**: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time (less than 15 minutes on a lab desktop), and you must follow all instructions provided below, including specific implementation requirements. Indexing or translation steps might taken longer than this, however you should upload with your submission preprocessed files (e.g., text files, pickled python structures) to keep the runtime modest.
You will be marked not only on the correctness of your methods, but also on your explanation and analysis. 

Please do not change any of instruction text in the notebook. Where applicable, leave the output cells in the code, particularly when you are commenting on that output. You should add your answers and code by inserting a markdown cell between every major function or other block of code explaining its purpose or anywhere a result needs to be discussed (see the class notebooks for examples). Note that even if you do something wrong, you might get partial credit if you explain it enough that we can follow your reasoning, whereas a fully correct assignment with no text commentary will not receive a passing score. You will not be marked directly on the performance of your final classifier, but each of the steps you take to build it should be reasonable and well-defended.

**Updates**: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

**Academic Misconduct**: For most people, collaboration will form a natural part of the undertaking of this project, and we encourage you to discuss it in general terms with other students. However, it is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s [Academic Misconduct policy](http://academichonesty.unimelb.edu.au/policy.html) where inappropriate levels of collusion or plagiarism are deemed to have taken place.


### Warning: File encodings and tokenisation

All the text files are encoded in *utf-8*, which requires some care in using with python strings, the nltk tools and jupyter.  Please use the following method to convert strings into ASCII by escaping the special symbols. Be careful to do the conversion after tokenisation, as the escaped tags might interfere with the NLTK tokenizer and get treated as punctuation.

# Part 1: CLIR engine

The project has two parts: first you must build a CLIR engine, comprising information retrieval and translation components, and evaluate its accuracy. Next you will propose an extension to a component of the system. In each step you will need to justify your modelling and implementation decisions, and evaluate the quality of the outputs of the system, and at the end you will need to reflect on the overall outcomes.

The CLIR system will involve:
 - translating queries from German into English, the language of our text collection, using word-based translation
 - once the queries and the documents are in the same language, search over the document collection using BM25
 - evaluate the quality of ranked retreival results using the query relevance judgements

Note that you could try translating the text collection into German instead, but for this project we'll stick with translating the query.

Building the CLIR engine is the majority of work in the assignment, and  constitutes 70% of the assignment mark. Note that although there are several steps, they do not necessarily need to be attempted in a linear order. That is, some components are independent of others. If you're struggling with one component and cannot complete it, then you may be able to skip over it and return to it later. Where outputs are needed in a subsequent step you may want to consider ways of side-stepping this need, e.g., by using Google translate output rather than your system output for translating the queries. You should aim to answer all components.

## Information Retreival with BM25 (20% of assignment mark)

You should implement a vector-space retreival system using the BM25 approach. You'll need to:
 - load in the data files and tokenize the input,
 - preprocess the lexicon, e.g., by stemming and stop-word removal, 
 - calculate the TF/IDF representation for all documents in the collection,
 - store an inverted index such that you can efficiently recover the documents matching a query term, and
 - implement querying in the BM25 model
 - test that it runs with some English queries (you'll have to make these up)
 
This should run in a reasonable time over *dev.docs* (up to a few minutes to index), but beyond this does not need to be highly optimised. Feel free to use python dictionaries, sets, lists, numpy/scipy matrices etc where appropriate for best runtime, but you shouldn't need to use specialised data structures such as compression schemes or the like. Feel free to use APIs in NLTK, scikit-learn and other allowed python modules as needed (see list above.) You will probably want to save and load your index to/from disk to save repeated re-indexing every time you load the file.

In [1]:
import nltk
import string
import collections
import math
from nltk.tokenize import word_tokenize
from six import iteritems
from six.moves import xrange

punctuation = set(string.punctuation)
english_stop_words = set(nltk.corpus.stopwords.words('english'))
german_stop_words = set(nltk.corpus.stopwords.words('german'))

porter_stemmer = nltk.stem.porter.PorterStemmer()

def tokenize(line, tokenizer = word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

def preprocess(doc):
    terms = []
    for token in tokenize(doc):
        if token not in punctuation and token not in english_stop_words:
            terms.append(token)
    return terms

def preprocess_de(doc):
    terms = []
    for token in tokenize(doc):
        if token not in punctuation and token not in german_stop_words:
            terms.append(token)
    return terms

def get_doc_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess(line))
    return querys

import pickle
def load_object(file_path):
    with open(file_path, 'rb') as f:
        object_instance = pickle.load(f)
    return object_instance

def saving_object(file_path, object_instance):
    with open(file_path, 'wb') as f:
        pickle.dump(object_instance, f, pickle.HIGHEST_PROTOCOL)

class PostingListConstructor(object):
    """docstring for PostingListConstructor"""
    def __init__(self, docs_path):
        super(PostingListConstructor, self).__init__()
        self.docs_path = docs_path
        self.posting_lists = {} # {term:(term_count, posting_list)}
        self.num_docs_with_term = {}
        self.len_docs = {} # {doc_id: _, doc_length: _}
        self.doc_id_list = []
        self.num_docs = 0
        self.total_doc_length = 0
        self.doc_wordcounters = []
        self.original_lines = []
        with open(docs_path, 'r') as f:
            lines = f.readlines()
            check_next = 0
            words = []
            for i in range(len(lines)):
                self.num_docs += 1
                doc_id, text = lines[i].split('\t')
                terms = []
                for token in tokenize(text):
                    if token not in punctuation and token not in english_stop_words:
                        terms.append(token)
                        words.append(token)
                        if (self.posting_lists.get(token, []) == []):
                            self.posting_lists[token] = self.posting_lists.get(token, []) + [i]
                        else:
                            if self.posting_lists[token][-1] != i:
                                self.posting_lists[token] = self.posting_lists.get(token, []) + [i]
                        if (self.num_docs_with_term.get(token, 0) == 0):
                            self.num_docs_with_term[token] = self.num_docs_with_term.get(token, 0) + 1
                        else:
                            if self.posting_lists[token][-1] != i:
                                self.num_docs_with_term[token] = self.num_docs_with_term.get(token, 0) + 1
                self.doc_id_list.append(doc_id)
                self.doc_wordcounters.append(collections.Counter(terms))
                self.original_lines.append(terms)
                self.len_docs[i] = len(terms)
                self.total_doc_length += len(terms)
        self.terms = list(set(words))

    def generate_td_list(self):
        terms = self.terms
        td_idf_list = []
        for term in terms:
            pointers = self.posting_lists[term]
            doc_freq = self.num_docs_with_term[term]
            idf = math.log(float(self.num_docs)/float(doc_freq))
            td_sub_list = []
            idf_list = []
            for pointer in pointers:
                td_sub_list.append((pointer, self.doc_wordcounters[pointer][term]*idf))
            td_idf_list.append((term, td_sub_list))
        td_idf = dict(td_idf_list)
        return td_idf

posting_list_generator = PostingListConstructor('./dev.docs')

In [2]:
class BM25(object):

    def __init__(self, corpus_constructor, k1 = 1.5, b= 0.75):
        self.corpus_size = corpus_constructor.num_docs
        self.avgdl = float(sum(corpus_constructor.len_docs.values())) / self.corpus_size
        self.posting_list_generator = corpus_constructor
        self.f = []
        self.df = {}
        self.idf = {}
        self.PARAM_K1 = k1
        self.PARAM_B = b
    def get_sent_score(self, querys):
        idf = []
        document_index = []
        document_scores = {}
        PARAM_K1 = self.PARAM_K1
        PARAM_B  = self.PARAM_B
        for query in querys:
            pointers = self.posting_list_generator.posting_lists.get(query, 0)
            if pointers != 0:
                document_index.extend(pointers)
        document_index = list(set(document_index))
        for i in document_index:
            scoredq = 0
            for query in querys:
                score = 0
                n = self.posting_list_generator.num_docs
                nqi = self.posting_list_generator.num_docs_with_term.get(query, 0)
                idfi = math.log((n-nqi+0.5) /(nqi+0.5))
                fqid = self.posting_list_generator.doc_wordcounters[i].get(query, 0)
                absd = self.posting_list_generator.len_docs[i]
                tfi = (fqid*(PARAM_K1+1))/(fqid+PARAM_K1*(1-PARAM_B+PARAM_B*absd/self.avgdl))
                scorei = idfi*tfi
                scoredq += scorei
            document_scores[i] = scoredq
        return document_scores

def get_sent_querys_dict(query_path, bm25model):
    querylist = []
    querys = get_doc_list(query_path)
    for query in querys:
        querylist.append(bm25model.get_sent_score(query))
    return querylist

import operator

bm25model = BM25(posting_list_generator)
querys = get_doc_list('./test_query')
querytest = get_sent_querys_dict('./test_query', bm25model)

for i in range(len(querytest)):
    max_index = max(querytest[i].iteritems(), key=operator.itemgetter(1))[0]
    print querys[i], max_index, posting_list_generator.original_lines[max_index]

['sweet', 'chocolate'] 935 ['chocolate', 'chocolate', 'processed', 'typically', 'sweetened', 'food', 'produced', 'seed', 'tropical', 'theobroma', 'cacao', 'tree', 'cacao', 'cultivated', 'least', 'three', 'millennia', 'mexico', 'central', 'america', 'northern', 'south', 'america', 'earliest', 'documented', 'use', 'around', '1100', 'bc', 'majority', 'mesoamerican', 'people', 'made', 'chocolate', 'beverages', 'including', 'aztecs', 'made', 'beverage', 'known', 'xocol\\u0101tl', 'nahuatl', 'word', 'meaning', 'bitter', 'water', 'seeds', 'cacao', 'tree', 'intense', 'bitter', 'taste', 'must', 'fermented', 'develop', 'flavor', 'fermentation', 'beans', 'dried', 'cleaned', 'roasted', 'shell', 'removed', 'produce', 'cacao', 'nibs', 'nibs', 'ground', 'cocoa', 'mass', 'pure', 'chocolate', 'rough', 'form', 'cocoa', 'mass', 'usually', 'liquefied', 'molded', 'without', 'ingredients', 'called', 'chocolate', 'liquor', 'liquor', 'also', 'may', 'processed', 'two', 'components', 'cocoa', 'solids', 'cocoa',

```The tfidf and bm25 algorithm are implemented according to the lecture material. I also use some machanism to remove the stop words and punctuation from nltk. I chose some typical types of query and test file to test how well the tfidf and bm25 works. The outcomes show that for the term or query which are highly relevant to a document, the tfidf and bm25 scores will be accordingly high. ```

## Translating the queries (40% of assignment mark)

For translation, you should implement a simple word-based translation model in a noisy channel setting, where you search for the best translation string using a language model over the English with a translation model over the back-translation of the English output string, $\vec{e}$, into the German input string, $\vec{f}$. This corresponds to finding the string, $\vec{e}$ which maximises $p(\vec{e}) p(\vec{f} | \vec{e})$. This has two components:

### Language model

The first step is to estimate a language model. You should learn both a unigram language model and a trigram language model, and compare the two. Note that the unigram will be used in the *decoding* step below. You will have to think about how to smooth your estimates, and the related problem of handling unknown words. For smoothing the trigram model, you should you use backoff smoothing. There are a few choices to be made about how much probability mass to assign to the backoff, you will need to decide how to do this -- see the lecture slides and readings for some ideas. Be careful to pad the sentence with sentinel symbols to denote the start and end of each sentence such that the model can predict the words at the start of a sentence, and those when a sentence is complete.

Please use *bitext-large.en* to train your models (start with *bitext-small.en* for your development) and evaluate the perplexity of your model on *newstest2013.en*. You should justify your development decisions and parameter settings in a markdown cell, and quantify their effect on perplexity, including the differences you notice between the unigram and the trigram models.

In [3]:
import nltk
import string
import collections
import math
from nltk.tokenize import word_tokenize
from six import iteritems
from six.moves import xrange

punctuation = set(string.punctuation)
english_stop_words = set(nltk.corpus.stopwords.words('english'))
german_stop_words = set(nltk.corpus.stopwords.words('german'))

porter_stemmer = nltk.stem.porter.PorterStemmer()


def tokenize(line, tokenizer = word_tokenize):
    utf_line = line.decode('utf-8').lower()
    return [token.encode('ascii', 'backslashreplace') for token in tokenizer(utf_line)]

def preprocess(doc):
    terms = []
    for token in tokenize(doc):
        if token not in punctuation and token not in english_stop_words:
            terms.append(token)
    return terms

def preprocess_de(doc):
    terms = []
    for token in tokenize(doc):
        if token not in punctuation and token not in german_stop_words:
            terms.append(token)
    return terms

def get_doc_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess(line))
    return querys

#this trigram with backoff smoothing
#with small bugs
from nltk.util import ngrams
PARAM_K = 0
def get_en_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess(line))
    return querys

def get_de_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess_de(line))
    return querys

#original list of list of token, after punctuation and stop word removal
origianl_docs = get_en_list('./bitext-large.en')

from collections import defaultdict

def check_for_unk_train(word,unigram_counts):
    if word in unigram_counts:
        return word
    else:
        unigram_counts[word] = 0
        return "UNK"

def convert_sentence_train(sentence,unigram_counts):
    return ["<s>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s>"]

def convert_tri_sentence_train(sentence,unigram_counts):
    return ["<s>","<s>"] + [check_for_unk_train(token.lower(),unigram_counts) for token in sentence] + ["</s>", "</s>"]

def get_bigram_counts(sentences):
    bigram_counts = defaultdict(dict)
    unigram_counts = {}
    for sentence in sentences:
        sentence = convert_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 1):
            bigram_counts[sentence[i]][sentence[i+1]] = bigram_counts[sentence[i]].get(sentence[i+1],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    token_count = float(sum(unigram_counts.values()))
    unigram_counts["</s>"] = unigram_counts["<s>"]
    return unigram_counts, bigram_counts, token_count

def get_trigram_counts(sentences):
    trigram_counts = defaultdict(dict)
    unigram_counts = {}
    for sentence in sentences:
        sentence = convert_tri_sentence_train(sentence, unigram_counts)
        for i in range(len(sentence) - 2):
            trigram_counts[(sentence[i], sentence[i+1])][sentence[i+2]] = trigram_counts[(sentence[i], sentence[i+1])].get(sentence[i+2],0) + 1
            unigram_counts[sentence[i]] = unigram_counts.get(sentence[i],0) + 1
    token_count = float(sum(unigram_counts.values()))
    unigram_counts["</s>"] = unigram_counts["<s>"]
    return unigram_counts, trigram_counts, token_count

from numpy.random import choice 

def generate_sentence(bigram_counts):
    sentence = ["<s>"]
    while sentence[-1] != "</s>":
        prev_token_counts = bigram_counts[sentence[-1]]
        bigram_probabilities = []
        total_counts = float(sum(prev_token_counts.values()))
        for token in prev_token_counts:
            bigram_probabilities.append(prev_token_counts[token]/total_counts)
        sentence.append(choice(prev_token_counts.keys(), p=bigram_probabilities))
    return " ".join([sentence[1].title()] + sentence[2:-1]).replace(" ,",",").replace(" .", ".")

docs_unigrams, docs_trigrams, docs_token_count = get_trigram_counts(origianl_docs)
docs_unigrams1, docs_bigrams, docs_token_count1 = get_bigram_counts(origianl_docs)
docs_uni_value_counts = collections.Counter(docs_unigrams1.values())

import math

def get_log_prob_interp(sentence,i, unigram_counts,bigram_counts, trigram_counts, token_count, bigram_lambda, trigram_lambda):
    tri_full_hit = trigram_counts.get((sentence[i-2],sentence[i-1]), 0)
    if tri_full_hit != 0:
        tri_full_hit = tri_full_hit.get(sentence[i], 0)
    return math.log(trigram_lambda*tri_full_hit/float(bigram_counts[sentence[i-2]].get(sentence[i-1], 1)) + 
        (1 - trigram_lambda) *
        (bigram_lambda*bigram_counts[sentence[i-1]].get(sentence[i],0)/float(unigram_counts[sentence[i-1]]) + 
                    (1 - bigram_lambda)*unigram_counts[sentence[i]]/token_count))

def check_for_unk_test(word,unigram_counts):
    if word in unigram_counts and unigram_counts[word] > 0:
        return word
    else:
        return "UNK"

def convert_sentence_test(sentence,unigram_counts):
    return ["<s>", "<s>"] + [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s>", "</s>"]

def convert_sentence_uni_test(sentence,unigram_counts):
    return ["<s>"] + [check_for_unk_test(word.lower(),unigram_counts) for word in sentence] + ["</s>"]

def get_sent_log_prob_interp(sentence, unigram_counts, bigram_counts, trigram_counts, token_count, bigram_lambda, trigram_lambda):
    sentence = convert_sentence_test(sentence, bigram_counts)
    return sum([get_log_prob_interp(sentence,i, unigram_counts,bigram_counts, trigram_counts, token_count, bigram_lambda, trigram_lambda) for i in range(2,len(sentence))])

sentence = "president".split()
print convert_sentence_test(sentence, docs_bigrams)
print get_sent_log_prob_interp(sentence, docs_unigrams1, docs_bigrams, docs_trigrams, docs_token_count1, 0.95, 0.95)

['<s>', '<s>', 'president', '</s>', '</s>']
0.517627693901


In [4]:
def calculate_perplexity(sentences,unigram_counts,bigram_counts, trigram_counts, token_count, smoothing_function, parameter1, parameter2):
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence) + 2 # have to consider the end token
        total_log_prob += smoothing_function(sentence,unigram_counts,bigram_counts, trigram_counts, token_count, parameter1, parameter2)
    return math.exp(-total_log_prob/test_token_count)

test_set = get_en_list('./newstest2013.en')

print "interpolation"
for bigram_lambda in [0.99,0.95,0.75,0.5,0.25,0.001]:
    for trigram_lambda in [0.99,0.95,0.75,0.5,0.25,0.001]:
        print bigram_lambda, trigram_lambda
        print calculate_perplexity(test_set,docs_unigrams,docs_bigrams,docs_trigrams, docs_token_count,get_sent_log_prob_interp,bigram_lambda, trigram_lambda)   

interpolation
0.99 0.99
113896.479707
0.99 0.95
32778.0248501
0.99 0.75
9756.73645335
0.99 0.5
6101.38281902
0.99 0.25
5005.81145674
0.99 0.001
8864.36562582
0.95 0.99
53786.4347866
0.95 0.95
15479.1630025
0.95 0.75
4607.57513867
0.95 0.5
2881.06821197
0.95 0.25
2362.38170456
0.95 0.001
3835.95939287
0.75 0.99
25997.6367834
0.75 0.95
7482.06220759
0.75 0.75
2227.30720943
0.75 0.5
1392.2200256
0.75 0.25
1138.82950407
0.75 0.001
1669.6583844
0.5 0.99
19797.9548929
0.5 0.95
5698.05484495
0.5 0.75
1696.56678969
0.5 0.5
1060.3655046
0.5 0.25
865.731940879
0.5 0.001
1223.09680385
0.25 0.99
17883.5967627
0.25 0.95
5147.33979207
0.25 0.75
1533.07905097
0.25 0.5
958.52023226
0.25 0.25
782.245196532
0.25 0.001
1092.80910284
0.001 0.99
21570.9974026
0.001 0.95
6209.01113357
0.001 0.75
1850.12284675
0.001 0.5
1157.85648081
0.001 0.25
946.733280036
0.001 0.001
1371.8881601


PARAM_K is the threshold of counts. For a particular gram, if C is greater than k, the backoff machanism will not be applied. If the count is less than this threshold, which means the occurrence of this gram is not much enough, so the backoff machanism will be applied, and a lower ngram machinsm will measure the probability of the few-occurrece grams. For the poor time complexity of my ngram model, I failed to evaluate the perplexity in the test set. However, I simplify the test set, and choose randomly 1000 lines from the test set. Due to the poor quality of the implementation of backoff, I decided to change the practical smoothing machanism of interpolation.

In the interpolation machanism, there are a parameter lambda. For trigram language model, there are 2 lambdas for trigram and bigram respectively.Shown as the outcomes above when the lambdas are both high, the perplexity is extremely high. It goes down with the decreas of the 2 lambdas. By doing the test, when the system get the lowest perplexity, the 2 lambdas were both 0.25. So in the next experiment, I decided to use these two figures.

For this part, only trigram language model has been implemented, and evaluated. For the unigram language model with good turing smoothing, I will evaluate it at the conclusive model of this module.

### Translation model

The next step is to estimate translation model probabilities. For this we will use a word-based alignment model, e.g., *ibm1* to learn word-based translation probabilities using expectation maxisation. Your task is to obtain good quality word alignments from this data, which will require careful use of the word alignment models. You will want to training in both translation directions, and combining the alignments.

You should use the *bitext-small* files for this purpose, and be aware that it may take a minute or two to train ibm1 on this data. Inspect some of the word alignments and translation probabilities (e.g., using a few common words in German such as *haus*) to see if your approach is working, and give some examples of output alignments that you find that are good and bad. 

In [5]:
from nltk.translate import IBMModel1
from nltk.translate import AlignedSent, Alignment

def get_en_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess(line))
    return querys

def get_de_list(doc_path):
    querys = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            querys.append(preprocess_de(line))
    return querys


en_text = get_en_list('./bitext-small.en')
de_text = get_de_list('./bitext-small.de')

all_en_words_set = set([item for sublist in en_text for item in sublist])
all_de_words_set = set([item for sublist in de_text for item in sublist])

all_en_words = list(all_en_words_set)
all_de_words = list(all_de_words_set)

#english to german
bitext_en_de = zip(en_text, de_text)
bt_en_de = [AlignedSent(E,F) for E,F in bitext_en_de]
imb1_en_de = IBMModel1(bt_en_de, 5)
tm_en_de = imb1_en_de.translation_table

#english to german
bitext_de_en = zip(de_text, en_text)
bt_de_en = [AlignedSent(E,F) for E,F in bitext_de_en]
imb1_de_en = IBMModel1(bt_de_en, 5)
tm_de_en = imb1_de_en.translation_table

print 'finished'
   

finished


In [6]:
print tm_en_de['area']
print tm_de_en['haus']

defaultdict(<function <lambda> at 0x000000011063E048>, {'chile': 0.00015166070253387994, 'business\\u201c': 0.0032871329181235367, 'b\\xfcndnisgrenzen': 0.10944188675423058, 'sieht': 5.6998985693486826e-05, 'grenzgebiet': 0.09707683355773049, '\\xf6l': 2.909611575055009e-06, 'unsicherheit': 9.576339660187786e-07, '\\u201ethe': 0.0036946244498101007, 'jahre': 2.4354254190197598e-08, 'entsprechend': 0.0004402240357246591, 'entdeckt': 0.00010177297677489431, 'bosnien': 1.8974044148007754e-05, 'letzten': 2.1904111696832166e-10, 'w\\xfcrden': 3.9746293100444196e-09, 'regierung': 1.4723692786175813e-11, 'wandels': 9.486447907150684e-06, 'au\\xdferhalb': 1.2582370601942222e-06, 'zeit': 5.773527511412098e-11, 'noten': 0.035044730588485666, 'bulgarien': 2.7603686530199758e-05, 'einziehen': 0.04129895213923631, '198': 0.019511727777349327, 'schlug': 0.005268808462240733, 'gerste': 0.06059668585656223, 'nieder': 0.0021310809484891813, 'pal\\xe4stinenser': 2.560950662439944e-05, 'entwicklung': 1.9

### Decoding 

Finally you'll need to solve for the best translation for a given string. For this you should do simple word-for-word translation where each word is translated independently. Use the alignments learned above (or the translation parameters) to translate each word of the queries into English. Compare taking the maximum probability of translation into English $p(e|f)$, with the noisy-channel probability, $p(f|e)p(e)$, which considers the reverse translation and a unigram language model probability. (You'll get a chance later to do something more ambitious, using the trigram model.)

Use your algorithms to translate the first 100 queries into English, and save them to disk.

In [7]:
from decimal import *

def get_log_uni_prob_gt(token, unigram_counts, value_counts, token_count, k):
    c = unigram_counts.get(token, 0)
    nc = value_counts.get(c, 1)
    nc1 = value_counts.get(c+1, 1)
    if c != 0:
        c_star = c
    else:
        c_star = (c+1)*nc1/nc
    p = Decimal(c_star)/Decimal(token_count)
    if k == 1:
        return float(c_star)/float(token_count)
    else:
        return p.ln()

def get_sent_uni_log_prob_gt(sentence, unigram_counts, value_counts, token_count):
    sentence = convert_sentence_test(sentence, unigram_counts)
    return sum([get_log_uni_prob_gt(sentence[i], unigram_counts, value_counts, token_count, 5) for i in range(0,len(sentence))])

def calculate_uni_perplexity(sentences,unigram_counts,token_count, smoothing_function):
    value_counts = collections.Counter(unigram_counts.values())
    total_log_prob = 0
    test_token_count = 0
    for sentence in sentences:
        test_token_count += len(sentence)+1 # have to consider the end token
        total_log_prob += smoothing_function(sentence,unigram_counts, value_counts, token_count)
    return math.exp(-total_log_prob/test_token_count)

print "Unigram Perplexity with Good Turing Smoothing: "
print calculate_uni_perplexity(test_set,docs_unigrams1,docs_token_count1, get_sent_uni_log_prob_gt)

Unigram Perplexity with Good Turing Smoothing: 
6932.66847706


In [8]:
def get_de_query_list(doc_path):
    querys = []
    query_id_list = []
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            doc_id, text = line.split('\t')
            querys.append(preprocess_de(text))
            query_id_list.append(doc_id)
    return query_id_list, querys

de_query_id, de_query_list = get_de_query_list("./dev.queries")

def decode(query, translation_model, unigram_counts, bigram_counts, trigram_counts, token_count, n):
    value_counts = collections.Counter(unigram_counts.values())
    final_result = []
    for de_token in query:
        string = ""
        prob = -999999
     
        for token, value in translation_model[de_token].iteritems():
            if((token != 'None') and token not in english_stop_words):
                uni_log_prob = get_log_uni_prob_gt(token, unigram_counts, value_counts, token_count, 1)
                prob_tmp = value * uni_log_prob
                if prob_tmp > prob:
                    prob = prob_tmp
                    string = token
        if string != "" and string != '``':
            final_result.append(string)
    return final_result

querys = de_query_list[0:100]
outcomes = []
for query in querys:
    outcomes.append(decode(query, tm_de_en, docs_unigrams1, docs_bigrams, docs_trigrams, docs_token_count1, 1))

import pickle
def load_object(file_path):
    with open(file_path, 'rb') as f:
        object_instance = pickle.load(f)
    return object_instance

def saving_object(file_path, object_instance):
    with open(file_path, 'wb') as f:
        pickle.dump(object_instance, f, pickle.HIGHEST_PROTOCOL)

print len(outcomes)
saving_object('./translated_100.en', outcomes)

100


Now you will need to write some text about the procedure above, and how well it worked. You should answer:
  - how good are the translations? How do your methods fare at translating the content words versus functions words? You can use Google translate to provide an alternative (pretty good) translation for comparison purposes. 
  - what are good settings for various modelling parameters such as language model discounting, translation model smoothing, any decoding parameters, and how do these affect the outputs?
  - compare the language model perplexity for the unigram and trigram models over your decoder outputs, how does the difference in perplexity between the two models compare to your test above on the monolingual data? 
  - how do the learned alignments differ between the two translation directions, and does their combination appear oto improve the predictions? (You don't need a formal evaluation here, as we have no *gold standard* alignments.)

For single query, the translator works well. It sometime can return a meaningful outcome. For example, searching for "verliert", it can return "loses", and "verbindungen" for "ties", although, tie is a uncommon translation of "verbindungen". For quite much test words or artifatically selected words, the translation model can give a pretty good translation, only if the word is a content word. Because of the stop word removal machanism, it cannot return translation for any functions words I tested.

For the trigram language model with interpolation smoothing method, I implemented above. There are 2 lambdas. For the selection of lambda, I experiment for a range of lambda for bigram and trigram. I found that when the lambdas are both 0.25, the perplexity is the least. For the unigram, the good turing smoothing does not have any parameter that can impact the final outcome.The perplexity is extremely high, which is aproximately 6932. It is much higher than trigram's.

The learned alignments performed pretty well. By observing the outcomes, their combination appears to improve the predictions. For the simple implementation, I only use a beginner alignments machanism.

## Putting the CLIR system together (10% of assignment mark)

Now you should couple the translation and IR components from above. Take your translated queries and use these with the IR system to find the most relevant documents. You should then evaluate these predictions against the supplied relevance judgements, using the *mean average precision* (MAP) metric. As part of this, you will want to consider tuning the settings of the IR system for best performance, and comment on how successful your approach was and evaluate the IR performance is affected by the parameter settings and modelling decisions you have made.

Note that if you were stuck on the translation steps above, or your translations are otherwise unusable, you can implement this step using google translate outputs for the 100 queries.

In [9]:
def get_query_result_dict(doc_path):
    query_result_dict = {}
    with open(doc_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            query_id, pad, result_id, score = line.split('\t')
            query_result_dict[query_id] = query_result_dict.get(query_id, []) + [result_id]
    return query_result_dict

query_result = get_query_result_dict('./dev.qrel')

In [13]:
tranlated_querys = load_object('./translated_100.en')
test_de_query_id = de_query_id[0: 100]

def get_query_precision(query_result_id, clir_results_index):
    query_result_id_set = set(query_result_id)
    tp = 0
    for index in clir_results_index:
        if posting_list_generator.doc_id_list[index] in query_result_id_set:
            tp += 1
    return float(tp)/len(query_result_id)

def get_sent_map(query_id, query_result, clir_results):
    sum_precision = 0.0
    for i in range(len(query_id)):
        precision = get_query_precision(query_result[query_id[i]], clir_results[query_id[i]])
        sum_precision += precision
    return sum_precision/len(de_query_id)

def display_tuning(kk1, bb):
    bm25model = BM25(posting_list_generator, k1 = kk1, b=bb)
    clir_results = {}
    for i in range(len(tranlated_querys)):
        relevant_doc_id = query_result[de_query_id[i]]
        num_results = len(relevant_doc_id)
        result_dict = bm25model.get_sent_score(tranlated_querys[i])
        sorted_result = sorted(result_dict.items(), key=operator.itemgetter(1), reverse = True)
        selected_clir_results = dict(sorted_result[0:num_results])
        clir_results[test_de_query_id[i]] = selected_clir_results.keys()
        
    print get_sent_map(test_de_query_id, query_result, clir_results)

for k1 in [0.5, 0.75, 1.25, 1.5, 2.0]:
    for b in [0.25, 0.5,0.75, 1.0]:
        print k1, b
        display_tuning(k1, b)

0.5 0.25
0.000666090392155
0.5 0.5
0.000665436797383
0.5 0.75
0.000668371425135
0.5 1.0
0.00066619835086
0.75 0.25
0.00068795266759
0.75 0.5
0.000682164438851
0.75 0.75
0.000683454563952
0.75 1.0
0.000678638292046
1.25 0.25
0.000754133422388
1.25 0.5
0.000750186411187
1.25 0.75
0.000711724690504
1.25 1.0
0.000727840535918
1.5 0.25
0.000748861923105
1.5 0.5
0.000756516966666
1.5 0.75
0.000726111721072
1.5 1.0
0.000696164878114
2.0 0.25
0.000752011766752
2.0 0.5
0.000753459551299
2.0 0.75
0.000729490982035
2.0 1.0
0.000694216382186


In [ ]:
saving_object('./translated_100.clir', clir_results)

The precision is the number of true positive divided by the number of true positive and false positive. However, the threshold or cretiria of relevance(how large the bm25 score should be means relevance)is unclear. So in the final test, I choose the number of CLIR result with most bm25 score same as the number of results read from dev.qrel. It will affact the final outcome. I assumed that the impact of this will be tiny. And the tuning of the IR module shows that when k1 is 1.5 and b is 0.5, the system gets the best mean average precision.

Note that in a complete CLIR you would translate the retreived documents back into German. We won't bother about that for this assignment, especially as I doubt many of you can understand German.

# Part 2: Extension and discussion

## Extension (25% of assignment mark)

You now have a working end-to-end CLIR system. The next step is to revisit some of the steps above to see if you can improve the performance of the system. You are invited to be creative with your ideas about how to do this. Note that many great ideas might not produce improvements, or end up being very difficult to implement. This will not be assessed solely on the grounds of CLIR accuracy; we will be looking primarily for interesting and creative ideas.

Here are some ideas on what you might do. Warning: some are bigger than others, aim to spend 5-6 hours in total on this.

#### Structured decoding

Decode using a higher order language model, possibly with word reordering. Using your trigram language model and a word-based or phrase-based translation model, search for the best scoring translation sentence. You may want to use the tools in *nltk.translate*, including the phrase extraction and *StackDecoder*, especially if you wish to support word reordering. The decoding algorithm is much simpler (and tractable) without reordering, so you might want to implement this yourself. You will want to work with log-probabilities to avoid numerical underflow. Note  that the *StackDecoder* is not highly stable code, so you will need to understand its code in detail if you chose to use it. You will need to supply translation log probabilities and language model log probabilities as input to the decoder and consider the context of the previous two words in estimating the probability of each word. 

#### Probabilisitc querying

Consider queries including weighted terms based on multiple translation outputs, such as the word translation probabilities or using *k-best* translations from a decoder (you will need to extract not just the best translation from the decode, but the runners up, which can be done from the stack contents after beam search.)  You should consider how best to define TF and DF for query words based on their ambiguous translations. You may find inspiration in the bibliography of [Douglas Oard](https://terpconnect.umd.edu/~oard/research.html#mlia) such as [this paper](https://terpconnect.umd.edu/~oard/pdf/coling12ture.pdf).

#### Word alignment

Implement an extended word alignment model, such as the *hmm* model (see [Vogel's paper](http://www.aclweb.org/anthology/C96-2141)) or a variant of *ibm2*. The variant of *ibm2* could include a better formulation for the distortion term, $p(i|j,I,J)$, based around rewarding values of i and j that are at similar relative positions to encourage alignments near the diagonal. This contrasts with *ibm2* which just learns a massive table of counts for all combinations of i,j,I and J. This strictly needs normalisation, as its not a probability, but for simplicity you can ignore this aspect here. This idea is inspired by [fast-align](https://github.com/clab/fast_align) which has a similar, but slightly more complex, formulation.

You might also want to experiment with [fast-align](https://github.com/clab/fast_align) (an extremely fast and accurate variant of *ibm2*) or [GIZA++](http://www.statmt.org/moses/giza/GIZA++.html) (an implementation of *ibm1* - *ibm5* and the *hmm*), both widely used and highly optimised alignment tools. Neither of these tools are in python, and may be a little involved to install and compile.

#### Decoding

Experiment with a state-of-the-art translation tool like [Moses](http://www.statmt.org/moses/).  Can you get better CLIR performance using this, trained on the full parallel dataset? You might want to consider translating the document collection into German, and then doing the IR entirely in German. You might consider building a python interface to moses, building on the existing very basic funcationality. **Warning:** Moses is a complex suite of software, and can be a little complex to install and use. However there are good installation and usage guides on the moses webpage.

#### IR engine

There are several great open source IR engines, including [Lucene](https://lucene.apache.org/core/), [Terrier](http://terrier.org/), [Zettair](http://www.seg.rmit.edu.au/zettair/) and [Lemur](http://www.lemurproject.org/). You could experiment with these, and use this to compare the accuracy of different types of IR models on the CLIR data. Alternatively, you could implement a faster or more compact retreival system in python, e.g., using compressed vbyte encodings, skip lists or similar.

Note that the development of some of the extensions maye require working in the command shell and in other languages than python. If this is the case, you should attach your code in separate files as part of your submission, but include the analysis text here. *Note that excellent submissions implementing interfaces or models not currently in NLTK will be considered for inclusion into the toolkit.* You should include both the code, and text explaining your work and findings.

```your supporting description analysis goes here```

## Discussion (5% of marks)

What conclusions can you make about CLIR, or more generally translation and IR? Overall what approaches worked and what ones didn't? What insights do you have from doing this, put another way, if you were to start again, what approach would you take and why?

Until here, the CLIR is finally completed and with a reasonable result set, although it is a beginner translation system. Many results cannot be a good translation even comparing to it given by Google Translation. Machine learning is based on CLIR, which takes an source query first, and then uses a range of language models, translation models and decoding methods to get a result in target language. This result is not a final work. It should still be retrieved in a huge size of corpus data. Due to the limitation of time, this CLIR system can not be well considered and constructed. For example, there are more effective decoding method, and also a range of methods of selecting threshold of relevance. If I start this again, I will use translation model with word order instead of ibm1, which has no word order. And trigram decoding method. 